In [ ]:
from get_urls import get_url_tool
from StackOverflow import Stack_overflow_tool
from summarizer import StackOverflowSummarizer
url=get_url_tool

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()
from groq import Client,Groq
llm=Groq()

In [3]:
from typing import Annotated
from typing_extensions import TypedDict
from langchain_core.messages import SystemMessage, ToolMessage, HumanMessage, AIMessage, AnyMessage
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages


class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]
    used_tool_ids: list[str]




class Agent:
    def __init__(self, model, tools, system=""):
        self.system = system
        self.model = model.bind_tools(tools)
        self.tools = {t.name: t for t in tools}
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_groq)

        # Dynamically add tool nodes and edges
        for tool in tools:
            node_name = tool.name  # Must match LLM's tool call name exactly
            graph.add_node(node_name, self.take_action_for(tool.name))
            graph.add_edge(node_name, "llm")

        # Conditional edges: map from tool name directly
        graph.add_conditional_edges(
            "llm",
            self.action_exists,
            {tool.name: tool.name for tool in tools} | {False: END}
        )

        graph.set_entry_point("llm")
        self.graph = graph.compile()

    def action_exists(self, state: AgentState):
        result = state['messages'][-1]
        if hasattr(result, "tool_calls") and result.tool_calls:
            for call in result.tool_calls:
                if call["id"] not in state.get("used_tool_ids", []) and call["name"] in self.tools:
                    return call["name"]
        return False


    def call_groq(self, state: AgentState):
        messages = state["messages"]
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {
            'messages': [message],
            'used_tool_ids': state.get('used_tool_ids', [])  # keep track
        }


    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f'Calling: {t}')
            result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t["id"], name=t["name"], content=str(result)))
        print("Back to the Model !!")
        return {"messages": results}

    # Generate a dedicated tool handler per tool
    def take_action_for(self, tool_name):
        def _handler(state: AgentState):
            tool_calls = state['messages'][-1].tool_calls
            results = []
            used_ids = state.get("used_tool_ids", [])
            for t in tool_calls:
                if t['name'] == tool_name and t['id'] not in used_ids:
                    print(f'Calling tool: {tool_name}')
                    result = self.tools[t['name']].invoke(t['args'])
                    results.append(ToolMessage(
                        tool_call_id=t["id"], name=t["name"], content=str(result)
                    ))
                    used_ids.append(t["id"])
            return {
                "messages": results,
                "used_tool_ids": used_ids
            }
        return _handler



In [ ]:
from langchain_groq import ChatGroq
prompt = """You MUST use the following tools in this exact order if the query is about programming:

1. get_url_tool
2. Stack_overflow_tool
3. StackOverflowSummarizer

for eg :
    user query : how to reverse a string in python ?
    
    1st call get url tool with the query to get the related urls.
    then call Stack_overflow_tool with the urls to get the related stackoverflow answers.
    then call StackOverflowSummarizer to summarize the stackoverflow answers.
    

Do NOT skip any tool. Do NOT change the order. Use one at a time in this order.
otherwise, if the question is not related to programming, just answer the question directly.
You must make tool calls either sequentially.
"""

tools = [get_url_tool, Stack_overflow_tool, StackOverflowSummarizer]
model = ChatGroq(model="llama-3.1-8b-instant")

abot = Agent(model, tools, prompt)

In [61]:
print(abot.graph.get_graph().draw_ascii())

                                              +-----------+                                        
                                              | __start__ |                                        
                                              +-----------+                                        
                                                     *                                             
                                                     *                                             
                                                     *                                             
                                                **+-----+...                                       
                                         *******  | llm |.  ......                                 
                                    *****        .+-----+ ..      .......                          
                             *******           ..           ...          ......                    


In [63]:
messages=[HumanMessage(content="How to reverse a string in Python?"),]
for event in abot.graph.stream({'messages':messages}):
    for v in event.values():
        print(v)

{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_bf2w', 'function': {'arguments': '{"query": "reverse a string in Python"}', 'name': 'get_url_tool'}, 'type': 'function'}, {'id': 'call_vv3a', 'function': {'arguments': '{"urls": ["https://stackoverflow.com/questions/3923216/how-to-reverse-a-string-in-python", "https://stackoverflow.com/questions/4034057/reverse-a-string-in-python"]}', 'name': 'stack_overflow_tool'}, 'type': 'function'}, {'id': 'call_fghc', 'function': {'arguments': '{"query": "reverse a string in Python", "stackoverflow_data": [{"question": "How to reverse a string in Python?", "answers": [{"Upvotes": 434, "Body": "You can use slicing to reverse a string in Python.\\nstring[::-1]", "Link": "https://stackoverflow.com/questions/3923216/how-to-reverse-a-string-in-python/3923230#3923230"}, {"Upvotes": 141, "Body": "Another way is to use the reversed function from the builtins module.\\nreversed(\'Hello World\')", "Link": "https://stackoverfl

In [4]:
from langchain_groq import ChatGroq

model = ChatGroq(model="llama-3.1-8b-instant")

llm_with_tools = model.bind_tools(tools=[get_url_tool, Stack_overflow_tool, StackOverflowSummarizer])
msg=llm_with_tools.invoke([HumanMessage(content="How to reverse a string in Python?")])
print(msg)

content='' additional_kwargs={'tool_calls': [{'id': 'call_wgc9', 'function': {'arguments': '{"query": "python reverse string"}', 'name': 'get_url_tool'}, 'type': 'function'}, {'id': 'call_gmpb', 'function': {'arguments': '{"urls": ["https://stackoverflow.com/questions/9319921/what-is-the-reverse-of-a-string-in-python"]}', 'name': 'stack_overflow_tool'}, 'type': 'function'}]} response_metadata={'token_usage': {'completion_tokens': 51, 'prompt_tokens': 1209, 'total_tokens': 1260, 'completion_time': 0.068, 'prompt_time': 0.061556196, 'queue_time': -0.379131956, 'total_time': 0.129556196}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_f7bd09b454', 'finish_reason': 'tool_calls', 'logprobs': None} id='run-75bbf876-42cb-48e5-ae10-05c063e7465f-0' tool_calls=[{'name': 'get_url_tool', 'args': {'query': 'python reverse string'}, 'id': 'call_wgc9', 'type': 'tool_call'}, {'name': 'stack_overflow_tool', 'args': {'urls': ['https://stackoverflow.com/questions/9319921/what-is-the-reve

In [5]:
msg.tool_calls

[{'name': 'get_url_tool',
  'args': {'query': 'python reverse string'},
  'id': 'call_wgc9',
  'type': 'tool_call'},
 {'name': 'stack_overflow_tool',
  'args': {'urls': ['https://stackoverflow.com/questions/9319921/what-is-the-reverse-of-a-string-in-python']},
  'id': 'call_gmpb',
  'type': 'tool_call'}]